In [15]:
import numpy as np
import mujoco
import time

def calibrate_effectiveness(model, data, drone_id, dt,
                            delta_u=0.05, settle_steps=50):
    """
    Calibra la matriz B tal que:
        alpha ≈ B * delta_u

    Retorna:
        B: (3x4) matriz de efectividad angular
        B_pinv: pseudoinversa
    """

    # Inicializar
    B = np.zeros((3, 4))

    # Estado base: todos los motores iguales (hover aproximado)
    u_base = np.ones(4) * 0.8
    data.ctrl[:] = u_base

    # Dejar estabilizar
    for _ in range(settle_steps):
        mujoco.mj_step(model, data)

    # Medir omega inicial
    quat = data.xquat[drone_id].copy()
    R = quat_to_rotmat(quat)
    omega0 = R.T @ data.cvel[drone_id, 0:3]

    for i in range(4):
        # Aplicar incremento solo al motor i
        u_test = u_base.copy()
        u_test[i] += delta_u
        data.ctrl[:] = u_test

        # Un paso
        mujoco.mj_step(model, data)

        # Medir omega nuevo
        quat = data.xquat[drone_id].copy()
        R = quat_to_rotmat(quat)
        omega1 = R.T @ data.cvel[drone_id, 0:3]

        # Aproximar aceleración angular
        alpha = (omega1 - omega0) / dt

        # Guardar columna
        B[:, i] = alpha / delta_u

        # Volver a base
        data.ctrl[:] = u_base
        mujoco.mj_step(model, data)

    B_pinv = np.linalg.pinv(B)
    return B, B_pinv

def quat_to_rotmat(q):
    w, x, y, z = q
    return np.array([
        [1 - 2*(y*y + z*z),     2*(x*y - z*w),     2*(x*z + y*w)],
        [    2*(x*y + z*w), 1 - 2*(x*x + z*z),     2*(y*z - x*w)],
        [    2*(x*z - y*w),     2*(y*z + x*w), 1 - 2*(x*x + y*y)]
    ])


XML_PATH = "models/single_quadrotor.xml"  # <-- tu archivo
model = mujoco.MjModel.from_xml_path(XML_PATH)
data = mujoco.MjData(model)

drone_id = mujoco.mj_name2id(
    model, mujoco.mjtObj.mjOBJ_BODY, "drone_3d"
)
dt = model.opt.timestep

print("Calibrando matriz INDI...")
B, B_pinv = calibrate_effectiveness(
    model, data, drone_id, dt,
    delta_u=0.05,
    settle_steps=100
)
print("B =", B)


Calibrando matriz INDI...
B = [[-4.63261936e-03 -1.88258669e-02  1.70173428e+02 -6.58765440e+01]
 [-1.70192190e+02  6.58546919e+01  3.99250217e+01  2.46807105e+01]
 [-2.99077197e+01  6.76329850e+00 -2.46984893e+01  1.08063589e+01]]


# INDI controller

In [ ]:
import time
import numpy as np
import mujoco
from mujoco import viewer


# ============================================================
# 1) Utilidades pequeñas
# ============================================================

def clamp(x, lo, hi):
    """Limita un escalar entre [lo, hi]."""
    return max(lo, min(hi, x))

def vee(M):
    """
    Operador vee para matrices skew-symmetric:
    [  0  -z   y
       z   0  -x
      -y   x   0 ]  -> [x, y, z]
    """
    return np.array([M[2, 1], M[0, 2], M[1, 0]])

def quat_to_rotmat(q):
    """Quaternion (w,x,y,z) -> matriz de rotación 3x3."""
    w, x, y, z = q
    R = np.array([
        [1 - 2*(y*y + z*z),     2*(x*y - z*w),     2*(x*z + y*w)],
        [    2*(x*y + z*w), 1 - 2*(x*x + z*z),     2*(y*z - x*w)],
        [    2*(x*z - y*w),     2*(y*z + x*w), 1 - 2*(x*x + y*y)]
    ])
    return R


# ============================================================
# 2) Disturbancia: fuerza externa desconocida (solo para sim)
# ============================================================

def unknown_force_world(pos):
    """
    Fuerza externa "desconocida" (en mundo).
    El controlador NO la conoce.
    """
    x, y, z = pos
    fx = 0.8 * np.sin(0.7 * x)
    fy = 0.8 * np.cos(0.7 * y)
    fz = 0.4 * np.sin(0.5 * z)
    # return np.array([fx, fy, fz])
    return np.array([0.0, 0.0, 0.0])  # solo en XY para ver efecto horizontal


# ============================================================
# 3) Trayectoria deseada (hover con leve excitación)
# ============================================================

def desired_trajectory(t, p0):
    # p_d = p0 + np.array([0.2*np.sin(0.4*t), 0.2*np.cos(0.4*t), 0.0])
    # v_d = np.array([0.2*0.4*np.cos(0.4*t), -0.2*0.4*np.sin(0.4*t), 0.0])
    # a_d = np.array([-0.2*(0.4**2)*np.sin(0.4*t), -0.2*(0.4**2)*np.cos(0.4*t), 0.0])

    p_d = p0 + np.array([-1.7*np.sin(0.3*t), 0.0, -0.5])
    v_d = np.array([-1.7*0.3*np.cos(0.3*t), 0.0, -0.0])
    a_d = np.array([1.7*(0.3**2)*np.sin(0.3*t), 0.0, -0.0])

    yaw_d = 0.0
    return p_d, v_d, a_d, yaw_d


# ============================================================
# 4) Outer loop (posición) -> fuerza deseada u_world (como paper)
#    u = kp e + kd e_dot + mg e3 + m a_d  (sin adaptación aquí)
# ============================================================

def position_outer_loop(pos, vel, p_d, v_d, a_d, m):
    g = 9.81
    e3 = np.array([0.0, 0.0, 1.0])

    # Ganancias sencillas (ajusta a gusto)
    kp = 6.0
    kd = 4.5

    # Errores estilo paper
    e = p_d - pos
    e_dot = v_d - vel

    # Fuerza deseada (mundo)
    u_world = kp*e + kd*e_dot + m*g*e3 + m*a_d
    return u_world


# ============================================================
# 5) Construir R_des a partir de u_world y yaw_d (parte tipo Mellinger)
#    Esto NO es el control de actitud; solo define la orientación deseada.
# ============================================================

def desired_attitude_from_force(u_world, yaw_d):
    u_norm = np.linalg.norm(u_world)
    if u_norm < 1e-6:
        u_norm = 1e-6

    # b3_des apunta en dirección de la fuerza deseada
    b3_des = u_world / u_norm

    # Vector “frontal” según yaw deseado en el plano XY
    b1_c = np.array([np.cos(yaw_d), np.sin(yaw_d), 0.0])

    # b2_des = b3 x b1_c
    b2_des = np.cross(b3_des, b1_c)
    if np.linalg.norm(b2_des) < 1e-6:
        b2_des = np.array([0.0, 1.0, 0.0])
    else:
        b2_des /= np.linalg.norm(b2_des)

    # b1_des = b2 x b3
    b1_des = np.cross(b2_des, b3_des)

    R_des = np.column_stack((b1_des, b2_des, b3_des))
    return R_des


# ============================================================
# 6) INDI (actitud): usa aceleración angular medida alpha_meas
#    Delta_tau = J (alpha_cmd - alpha_meas)
# ============================================================

def indi_attitude_increment(R, R_des, omega_body, alpha_meas, J):
    """
    Devuelve delta_tau (N*m) usando INDI.

    Idea:
      - Convertimos error de actitud a una "aceleración angular deseada" alpha_cmd
      - Comparamos con alpha_meas (medida) -> delta_tau = J*(alpha_cmd - alpha_meas)
    """

    # Limitar alpha_meas para evitar explosiones
    alpha_meas = np.clip(alpha_meas, -50.0, 50.0)

    # Ganancias en términos de aceleración (rad/s^2)
    # (si subes mucho, oscila)
    kR = 4.0
    kW = 1.2

    # Error geométrico de actitud:
    # e_R = 0.5 vee(R_des^T R - R^T R_des)
    e_R_mat = 0.5 * (R_des.T @ R - R.T @ R_des)
    e_R = vee(e_R_mat)

    # Error de velocidad angular (omega_des = 0 en este ejemplo)
    e_w = omega_body

    # Aceleración angular deseada (simple)
    alpha_cmd = -kR * e_R - kW * e_w

    # INDI: incremento de torque
    delta_tau = J @ (alpha_cmd - alpha_meas)
    return delta_tau


# ============================================================
# 7) Asignación a motores (allocator) para quad (plus) sencillo
#    Motor cmds u0..u3 -> [T, tau_x, tau_y, tau_z]
# ============================================================

def allocation_matrix(l=0.16, kf=1.0, km=0.05):
    """
    Matriz A tal que:
      [T, tau_x, tau_y, tau_z]^T = A * [u0,u1,u2,u3]^T

    - l: brazo (m) (aprox)
    - kf: factor thrust (en unidades de tu ctrl)
    - km: factor yaw moment (en unidades de tu ctrl)
    """
    # Convención plus:
    # tau_x: roll  -> diferencia entre motores laterales (2 y 3)
    # tau_y: pitch -> diferencia entre motores front/back (0 y 1)
    # tau_z: yaw   -> combinación alternada
    A = np.array([
        [kf,    kf,    kf,    kf],
        [0.0,   0.0,  l*kf, -l*kf],
        [-l*kf, l*kf, 0.0,   0.0],
        [-km,   km,   -km,    km]
    ], dtype=float)
    return A

def allocate_motors(T_des, tau_inc, u_base, A_pinv):
    """
    - T_des: thrust total deseado (N o escala)
    - tau_inc: incremento de torque (3,)
    - u_base: comando base de motores (4,)
    - A_pinv: pseudo-inversa de A

    Estrategia:
      - Mantener thrust con un base (u_base)
      - Sumar un incremento de motores para realizar delta_tau
    """
    # Solo pedimos incremento en torques (T_inc = 0)
    y_inc = np.array([0.0, tau_inc[0], tau_inc[1], tau_inc[2]])
    du = A_pinv @ y_inc

    # Ajustar base para el thrust deseado:
    # Queremos A*u_base tenga T cercano a T_des.
    # Aquí hacemos algo simple: repartir thrust entre 4 motores.
    u_th = (T_des / 4.0) * np.ones(4)

    # Control final
    u = u_th + du
    return u


# ============================================================
# 8) Main: simulación MuJoCo + outer loop + INDI
# ============================================================

def main():
    XML_PATH = "models/single_quadrotor.xml"  # <-- tu archivo
    model = mujoco.MjModel.from_xml_path(XML_PATH)
    data = mujoco.MjData(model)

    dt = model.opt.timestep

    # ID del body del dron
    drone_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "drone_3d")

    # Masa e inercia (aprox)
    mujoco.mj_forward(model, data)
    m = float(model.body_subtreemass[drone_id])
    J_diag = model.body_inertia[drone_id].copy()
    J = np.diag(J_diag)

    # Referencia hover
    p0 = data.xpos[drone_id].copy()

    # Para medir aceleración angular alpha_meas = (omega_k - omega_{k-1})/dt
    prev_omega_body = np.zeros(3)
    alpha_filt = np.zeros(3)
    alpha_filter = 0.1  # 0..1 (más alto = menos filtrado)

    # Matriz de asignación y pseudo-inversa
    A = allocation_matrix(l=0.16, kf=1.0, km=0.05)
    A_pinv = np.linalg.pinv(A)

    # Limites de actuadores (tu XML dice 0..2)
    CTRL_MIN = 0.0
    CTRL_MAX = 2.0

    # Reloj tiempo real (para que no vaya rápido)
    next_time = time.time()
    t = 0.0

    with viewer.launch_passive(model, data) as v:
        while v.is_running():
            # ------------------------------------------------
            # (1) Estado del dron
            # ------------------------------------------------
            pos = data.xpos[drone_id].copy()

            omega_world = data.cvel[drone_id, 0:3].copy()
            vel_world   = data.cvel[drone_id, 3:6].copy()

            quat = data.xquat[drone_id].copy()
            R = quat_to_rotmat(quat)

            # omega en body
            omega_body = R.T @ omega_world

            # ------------------------------------------------
            # (2) Disturbancia real (solo simulación)
            # ------------------------------------------------
            f_unk = unknown_force_world(pos)
            data.xfrc_applied[drone_id, 0:3] = 0.0
            data.xfrc_applied[drone_id, 3:6] = f_unk

            # ------------------------------------------------
            # (3) Referencia
            # ------------------------------------------------
            p_d, v_d, a_d, yaw_d = desired_trajectory(t, p0)

            # ------------------------------------------------
            # (4) Outer loop: fuerza deseada u_world
            # ------------------------------------------------
            u_world = position_outer_loop(pos, vel_world, p_d, v_d, a_d, m)

            # Thrust deseado (escala): proyección sobre b3 actual
            b3 = R[:, 2]
            T_des = float(u_world.dot(b3))
            T_des = max(0.0, T_des)  # thrust no puede ser negativo

            # ------------------------------------------------
            # (5) Orientación deseada a partir de u_world
            # ------------------------------------------------
            R_des = desired_attitude_from_force(u_world, yaw_d)

            # ------------------------------------------------
            # (6) Medir aceleración angular alpha_meas (dif finita + filtro)
            # ------------------------------------------------
            alpha_meas = (omega_body - prev_omega_body) / dt
            prev_omega_body = omega_body.copy()

            alpha_filt = (1 - alpha_filter)*alpha_filt + alpha_filter*alpha_meas

            # ------------------------------------------------
            # (7) INDI: incremento de torque
            # ------------------------------------------------
            tau_inc = indi_attitude_increment(R, R_des, omega_body, alpha_filt, J)

            # Limitar incremento para evitar explosiones
            tau_inc = np.clip(tau_inc, -0.2, 0.2)

            # ------------------------------------------------
            # (8) Allocate: thrust base + incremento torque
            # ------------------------------------------------
            u = allocate_motors(T_des, tau_inc, u_base=None, A_pinv=A_pinv)

            # Saturar por límites del XML
            u = np.clip(u, CTRL_MIN, CTRL_MAX)
            data.ctrl[:] = u

            # ------------------------------------------------
            # (9) Step simulación
            # ------------------------------------------------
            mujoco.mj_step(model, data)
            v.sync()

            # ------------------------------------------------
            # (10) Tiempo + tiempo real
            # ------------------------------------------------
            t += dt
            next_time += dt
            time.sleep(max(0.0, next_time - time.time()))


if __name__ == "__main__":
    main()


Calibrando matriz INDI B (3x4)...
B =
 [[-1.85551639e-08  8.83383850e-09  2.83145274e-01 -3.20243423e-01]
 [-2.83144948e-01  3.20243175e-01  1.48932907e-02 -1.81213232e-02]
 [-9.63117264e-02  9.03219393e-02 -9.10952299e-02  9.09954804e-02]]


# INDI callibrator